In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import itertools
plt.rcParams['figure.figsize'] = (12.0, 6.0)

In [20]:
# parameters
#settings = ['LAN', 'WAN']
settings = ['LAN']
parties = [0,1]
protocols = ['scs', 'poly']

all_params = itertools.product(settings, parties, protocols)

#data_filenames=['p0_lan.out', 'p0_wan.out', 'p1_lan.out', 'p1_wan.out']
data_filenames=['p0_lan.out', 'p1_lan.out']
raw_data = {}
pir_type = ""
k_A, k_B, l, n, time = 0., 0., 0., 0., 0.
for i_setting, setting in enumerate(settings):
    for p in parties:
        with open(data_filenames[p*len(settings) + i_setting]) as f:
            for line in f.read().splitlines():
                fields = line.split()
                if fields[0] == "Running":
                    pir_type = fields[-1][1:-1]
                    if (setting, p, pir_type) not in raw_data:
                        raw_data[(setting, p, pir_type)] = []
                if fields[0] == "l":
                    l = fields[-1]
                if fields[0] == "n":
                    n = fields[-1]
                if fields[0] == "k_A":
                    k_A = fields[-1]
                if fields[0] == "k_B":
                    k_B = fields[-1]
                if fields[0] == "Sparse":
                    time = fields[-2]
                    raw_data[(setting, p, pir_type)] += \
                        [[k_A, k_B, l, n, time]]

data = {param : np.array(raw_data[param], dtype=np.float64) for param in raw_data.keys()}

# data is (k_A, k_B, l, n)
f_mpc = { # same for both parties
    'scs': lambda data,a,b,c,d,e: [a*n + b*n*np.log2(n) + \
        d #c*n*(np.log2(n)**2) + d
            for n in [data[:,0] + data[:,1]]][0] + \
        e * (data[:,0] + data[:,1]) * data[:,2] * data[:,3],
    'poly': lambda data,a,b,c,d,e: a * data[:,0] * data[:,1] + \
        b * data[:,0] + c * data[:,1] + d + \
        e * (data[:,0] + data[:,1]) * data[:,2] * data[:,3]
}

coeffs = {
    params : curve_fit(f_mpc[params[2]], data[params][:,:4], data[params][:,4])[0]
    for params in data.keys()
}

def model(params):
    (setting, p, algo) = params
    return lambda data: f_mpc[algo](data, *(coeffs_mpc[params]))

/usr/lib/python3.6/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
